<a href="https://colab.research.google.com/github/pinzar14/Procesamiento_digital_de_imagenes/blob/main/2PD_Seniales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###-2 Tarea. Procesamiento digital de señales

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, VBox, interactive_output
from IPython.display import display
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram

In [2]:


# Función de actualización
def update_window(start_time, window_duration, t_total, fs, f_start, f_end):
    # Recalcular el tiempo y la señal en función de los nuevos valores
    t = np.linspace(0, t_total, int(fs * t_total))
    k = np.log(f_end / f_start) / t_total
    phi_t = 2 * np.pi * (f_start / k) * (np.exp(k * t) - 1)
    signal = np.sin(phi_t)
    signal = signal / np.max(np.abs(signal))

    start_sample = max(0, int(start_time * fs))
    end_sample = min(len(signal), start_sample + int(window_duration * fs))

    t_window = t[start_sample:end_sample]
    win_signal = signal[start_sample:end_sample]

    # Calcular FFT
    N = len(win_signal)
    freqs = fftfreq(N, 1/fs)[:N//2]
    spectrum = np.abs(fft(win_signal))[:N//2]

    # Frecuencia máxima con energía significativa (>1% del pico)
    threshold = 0.01 * np.max(spectrum)
    max_freq_idx = np.where(spectrum > threshold)[0]
    f_max_window = freqs[max_freq_idx[-1]] if len(max_freq_idx) > 0 else fs/2

    # Ajustar nperseg en función de la duración de la ventana
    nperseg = min(max(256, int(window_duration * fs / 2)), 1024)

    # Espectrograma solo sobre la ventana
    f_spec, t_spec, Sxx = spectrogram(win_signal, fs=fs, nperseg=nperseg, noverlap=nperseg // 2)
    Sxx_dB = 10 * np.log10(Sxx + 1e-10)

    # Ajustar rango dB local
    dB_min = np.min(Sxx_dB)
    dB_max = np.max(Sxx_dB)

    # Graficar todo en una sola figura
    fig, axs = plt.subplots(4, 1, figsize=(10, 14))  # Agregar una gráfica más para la señal

    # Señal completa y ventana seleccionada
    axs[0].plot(t, signal, alpha=0.3, label='Señal completa')
    axs[0].plot(t_window, win_signal, label='Ventana')
    axs[0].axvspan(t_window[0], t_window[-1], color='yellow', alpha=0.3)
    axs[0].set_title('Señal y ventana seleccionada')
    axs[0].set_xlabel('Tiempo [s]')
    axs[0].set_ylabel('Amplitud')
    axs[0].legend()
    axs[0].grid()

    # Nueva gráfica para mostrar solo la parte seleccionada de la señal
    axs[1].plot(t_window, win_signal, label='Señal en la ventana')
    axs[1].set_title('Parte de la Señal en la Ventana Seleccionada')
    axs[1].set_xlabel('Tiempo [s]')
    axs[1].set_ylabel('Amplitud')
    axs[1].legend()
    axs[1].grid()

    # FFT de la ventana
    axs[2].plot(freqs, spectrum)
    axs[2].set_xlim(0, f_max_window * 1.1)
    axs[2].set_title('FFT de la ventana')
    axs[2].set_xlabel('Frecuencia [Hz]')
    axs[2].set_ylabel('Magnitud')
    axs[2].grid()

    # Espectrograma de la ventana
    pcm = axs[3].pcolormesh(t_spec + t_window[0], f_spec, Sxx_dB, shading='gouraud', vmin=dB_min, vmax=dB_max)
    axs[3].axvspan(t_window[0], t_window[-1], color='yellow', alpha=0.3)
    axs[3].set_xlim(t_window[0], t_window[-1])
    axs[3].set_ylim(0, min(f_max_window * 1.1, fs / 2))
    axs[3].set_title('Espectrograma de la ventana')
    axs[3].set_xlabel('Tiempo [s]')
    axs[3].set_ylabel('Frecuencia [Hz]')
    axs[3].grid()

    # Barra de color
    cbar = fig.colorbar(pcm, ax=axs[3])
    cbar.set_label('Nivel [dB]')

    plt.tight_layout()
    plt.show()



In [3]:
# Sliders
start_slider = FloatSlider(value=0.0, min=0.0, max=20 - 0.1, step=0.01, description='Inicio (s)')
duration_slider = FloatSlider(value=1.0, min=0.1, max=10, step=0.01, description='Duración (s)')

# Sliders para t_total, fs, f_start y f_end
t_total_slider = FloatSlider(value=10, min=1, max=20, step=0.1, description='Tiempo Total (s)')
fs_slider = FloatSlider(value=8000, min=1000, max=20000, step=100, description='Frecuencia de muestreo (Hz)')
f_start_slider = FloatSlider(value=10, min=1, max=1000, step=1, description='Frecuencia inicio (Hz)')
f_end_slider = FloatSlider(value=15000, min=1001, max=20000, step=100, description='Frecuencia fin (Hz)')

# Unir los sliders en un VBox
ui = VBox([start_slider, duration_slider, t_total_slider, fs_slider, f_start_slider, f_end_slider])

# Conectar sliders con función
out = interactive_output(update_window, {
    'start_time': start_slider,
    'window_duration': duration_slider,
    't_total': t_total_slider,
    'fs': fs_slider,
    'f_start': f_start_slider,
    'f_end': f_end_slider
})

# Mostrar la interfaz de usuario
display(ui, out)



Output()